# Using Yahoo Finance to calc RSI and MFI

## Imports and Constants

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import mplfinance as mpf

In [3]:
candle_period = '1wk'
rolling_period = 14

## Get OHLCV

In [36]:
data = yf.download("SPY", interval = candle_period)

[*********************100%%**********************]  1 of 1 completed


In [5]:
type(data)

pandas.core.frame.DataFrame

In [6]:
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,1616.000000,1616.000000,1616.000000,1616.000000,1616.000000,1.616000e+03
mean,170.661637,173.299145,167.923609,170.913444,142.533669,4.074347e+08
std,106.957537,108.552689,105.453779,107.238226,113.073808,4.315150e+08
min,43.343750,43.968750,42.812500,43.562500,24.628672,1.022000e+05
25%,105.296875,107.615002,103.154688,105.572500,70.726841,4.792588e+07
50%,132.000000,134.178749,129.635002,132.019997,93.450745,3.153046e+08
75%,210.430000,212.300003,207.884995,210.312500,180.937065,5.602494e+08
max,476.299988,479.980011,473.299988,476.559998,476.559998,3.281576e+09


In [7]:
data.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-12-11,459.690002,473.730011,459.470001,469.329987,467.434814,486953100
2023-12-18,470.980011,475.899994,467.820007,473.649994,473.649994,382852200
2023-12-25,474.070007,477.549988,473.299988,475.309998,475.309998,322779500
2024-01-01,472.160004,473.670013,466.429993,467.920013,467.920013,397502600
2024-01-08,468.429993,477.450012,468.299988,476.559998,476.559998,274418260


## RSI

In [37]:
data['Change'] = data['Adj Close'].diff()
data['Gain'] = data['Change'].mask(data['Change'] < 0, 0.0)
data['Loss'] = -data['Change'].mask(data['Change'] > 0, -0.0)

In [38]:
data['Avg Gain'] = data['Gain'].rolling(rolling_period).mean()
data['Avg Loss'] = data['Loss'].rolling(rolling_period).mean()

In [54]:
data['Avg Gain'] = data['Gain'].ewm(com=rolling_period, adjust=False).mean()
data['Avg Loss'] = data['Loss'].ewm(com=rolling_period, adjust=False).mean()

In [55]:
data['RS'] = data['Avg Gain'] / data['Avg Loss']
data['RSI'] = 100 - (100 / (1 + data['RS']))
data

,Open,High,Low,Close,Adj Close,Volume,Change,Gain,Loss,Avg Gain,Avg Loss,RS,RSI
Date,,,,,,,,,,,,,
1993-01-25,43.968750,43.968750,43.750000,43.937500,24.840681,1003200,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-01,43.968750,45.093750,43.968750,44.968750,25.423714,2234800,0.583033,0.583033,0.000000,0.583033,0.000000,inf,100.000000
1993-02-08,44.968750,45.125000,44.531250,44.593750,25.211697,1159800,-0.212017,0.000000,0.212017,0.544164,0.014134,38.499055,97.468294
1993-02-15,44.468750,44.468750,42.812500,43.562500,24.628668,998700,-0.583029,0.000000,0.583029,0.507886,0.052061,9.755643,90.702555
1993-02-22,43.687500,44.437500,43.468750,44.406250,25.105692,1024300,0.477024,0.477024,0.000000,0.505829,0.048590,10.410131,91.235859
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-11,459.690002,473.730011,459.470001,469.329987,467.434814,486953100,9.093109,9.093109,0.000000,4.563744,2.291208,1.991851,66.575873
2023-12-18,470.980011,475.899994,467.820007,473.649994,473.649994,382852200,6.215179,6.215179,0.000000,4.673840,2.138461,2.185609,68.608832
2023-12-25,474.070007,477.549988,473.299988,475.309998,475.309998,322779500,1.660004,1.660004,0.000000,4.472918,1.995897,2.241057,69.145865


In [ ]:
mpf.plot(data, type='candle', style='yahoo', volume=True)

In [ ]:
data[['RSI']].iloc[-52:].plot()

## MFI

In [ ]:
data['Typical Price'] = (data['Adj Close'] + data['High'] + data['Low']) / 3
data['Money Flow'] = data['Typical Price'] * data['Volume']

In [ ]:
def mf_dir(typical_price,money_flow):

    positive_flow = []
    negative_flow = []
    
    for i in range(1, len(typical_price)):
        if typical_price[i] > typical_price[i-1]:
            positive_flow.append(money_flow[i-1])
            negative_flow.append(0)
            
        elif typical_price[i] < typical_price[i-1]:
            negative_flow.append(money_flow[i-1])
            positive_flow.append(0)
            
        else:
            positive_flow.append(0)
            negative_flow.append(0)
            
    return positive_flow, negative_flow

In [ ]:
data['Positive Flow'], data['Negative Flow'] = np.nan, np.nan

In [ ]:
data['Positive Flow'].iloc[1:],data['Negative Flow'].iloc[1:] = mf_dir(data['Typical Price'].to_numpy(), data['Money Flow'].to_numpy())

In [ ]:
data['Positive MF'] = data['Positive Flow'].rolling(rolling_period).sum()
data['Negative MF'] = data['Negative Flow'].rolling(rolling_period).sum()

In [ ]:
data['MFI'] = data['Positive MF']/ (data['Positive MF'] + data['Negative MF'])
data.tail()

In [ ]:
data[['MFI']].iloc[-52:].plot()